**Author:** Sirin Rabah - 324264589

In [54]:
%reload_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2, l1



Load The MNIST Dataset

In [95]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_of_classes = 10
image_shape = (28, 28, 1)

Normalize images function

In [96]:
def normalize_img(image, label):
    """Normalizes images: uint8 -> float32."""
    return tf.cast(image, tf.float32) / 255., label

Preprocess the training dataset

In [97]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(64)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
        normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

The neural network

In [118]:
model = tf.keras.Sequential([

    Flatten(input_shape=image_shape),

    #89
    Dense(89, activation='relu'),
    BatchNormalization(),

    #45
    Dense(45, activation='relu'),
    BatchNormalization(),

    #26
    Dense(26, activation='relu'),
    BatchNormalization(),

    #24
    Dense(24, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),

    Dense(10, activation='softmax')
])

In [119]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


Display model summary

In [120]:
model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_28 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_126 (Dense)                    │ (None, 89)                  │          69,865 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_88               │ (None, 89)                  │             356 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_127 (Dense)                    │ (None, 45)                  │           4,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_89               │ (None, 45)                  │             180 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_128 (Dense)                    │ (None, 26)                  │           1,196 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_90               │ (None, 26)                  │             104 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_129 (Dense)                    │ (None, 24)                  │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_77 (Dropout)                 │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_91               │ (None, 24)                  │              96 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_130 (Dense)                    │ (None, 10)                  │             250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 76,745 (299.79 KB)

 Trainable params: 76,377 (298.35 KB)

 Non-trainable params: 368 (1.44 KB)

Train the model

In [121]:
model.fit(ds_train,
          epochs=10,
          validation_data=ds_test)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.8507 - sparse_categorical_accuracy: 0.7450 - val_loss: 0.1652 - val_sparse_categorical_accuracy: 0.9526
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.2370 - sparse_categorical_accuracy: 0.9397 - val_loss: 0.1415 - val_sparse_categorical_accuracy: 0.9584
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.1890 - sparse_categorical_accuracy: 0.9518 - val_loss: 0.1242 - val_sparse_categorical_accuracy: 0.9650
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1638 - sparse_categorical_accuracy: 0.9587 - val_loss: 0.1017 - val_sparse_categorical_accuracy: 0.9712
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1472 - sparse_categorical_accuracy: 0.9635 - val_loss: 0.0970 - val_sparse_categorical_accuracy: 0.9724
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.1254 - sparse_categorical_accuracy: 0.9683 - val_loss: 0.0998 - val_sparse_categorical_accuracy: 0.9719
Epoc